# Tidying Eviction Data 

---
Like a lot of governmental data, this set has redundant an useless data. Here we set about tidying the data such that it may be easier to view and use. 

- Rename columns to remove spaces in label names
- Convert dates to datetime
- Remove 2017 data, as it only contains 3 months of the year. 
- Remove any rows with "null" Neighborhoods, as that data is useless to us.
- Remove rows that we will not end up using 
    - City
    - State 
    - Supervisor District
    - Location
    - Eviction ID
- Convert neighborhoods to lowercase, for later use with the police data 

In [4]:
import pandas as pd

In [16]:
evictions_raw = pd.read_csv('/data/leils/Eviction_Notices.csv')
list(evictions_raw)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Eviction ID',
 'Address',
 'City',
 'State',
 'Eviction Notice Source Zipcode',
 'File Date',
 'Non Payment',
 'Breach',
 'Nuisance',
 'Illegal Use',
 'Failure to Sign Renewal',
 'Access Denial',
 'Unapproved Subtenant',
 'Owner Move In',
 'Demolition',
 'Capital Improvement',
 'Substantial Rehab',
 'Ellis Act WithDrawal',
 'Condo Conversion',
 'Roommate Same Unit',
 'Other Cause',
 'Late Payments',
 'Lead Remediation',
 'Development',
 'Good Samaritan Ends',
 'Constraints Date',
 'Supervisor District',
 'Neighborhoods - Analysis Boundaries',
 'Location']

In [17]:
#Create a dataframe for tidy data  

evictions_tidy = evictions_raw[['Address',
                                'Non Payment',
                                 'Breach',
                                 'Nuisance',
                                 'Illegal Use',
                                 'Failure to Sign Renewal',
                                 'Access Denial',
                                 'Unapproved Subtenant',
                                 'Owner Move In',
                                 'Demolition',
                                 'Capital Improvement',
                                 'Substantial Rehab',
                                 'Ellis Act WithDrawal',
                                 'Condo Conversion',
                                 'Roommate Same Unit',
                                 'Other Cause',
                                 'Late Payments',
                                 'Lead Remediation',
                                 'Development',
                                 'Good Samaritan Ends',
                                 'Neighborhoods - Analysis Boundaries']]
evictions_tidy['date'] = pd.to_datetime(evictions_raw['File Date'])
evictions_tidy = evictions_tidy.rename(columns={'Neighborhoods - Analysis Boundaries': "Neighborhood"})
evictions_tidy['Neighborhood'] = evictions_tidy.Neighborhood.str.lower()
evictions_tidy = evictions_tidy.rename(columns=lambda x: x.replace(" ", ""))

evictions_tidy.head()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Address,NonPayment,Breach,Nuisance,IllegalUse,FailuretoSignRenewal,AccessDenial,UnapprovedSubtenant,OwnerMoveIn,Demolition,...,EllisActWithDrawal,CondoConversion,RoommateSameUnit,OtherCause,LatePayments,LeadRemediation,Development,GoodSamaritanEnds,Neighborhood,date
0,300 Block Of Park Street,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,bernal heights,2016-06-06
1,200 Block Of Seneca Avenue,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,outer mission,2016-06-29
2,200 Block Of Dolores Street,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,castro/upper market,2016-06-28
3,1200 Block Of 9th Avenue,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,inner sunset,2016-06-02
4,1400 Block Of Larkin Street,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,nob hill,2016-02-28


In [18]:
evictions_tidy.size

813780

In [19]:
#Removing null neighborhoods and dates, as they are useless to us. 
evictions_tidy = evictions_tidy.dropna(subset=['Neighborhood', 'date'], how='any')
evictions_tidy.size

782034

In [20]:
#Remove 2017 data 
evictions_tidy = evictions_tidy[evictions_tidy.date.dt.year != 2017]
evictions_tidy.size

779944

In [47]:
evictions_tidy.to_csv('/data/leils/tidy_evictions.csv', index=False, index_label=False)

In [24]:
evictions_mission = evictions_tidy[evictions_tidy.Neighborhood == 'mission']
del evictions_mission['Address']
evictions_mission.head()

,NonPayment,Breach,Nuisance,IllegalUse,FailuretoSignRenewal,AccessDenial,UnapprovedSubtenant,OwnerMoveIn,Demolition,CapitalImprovement,...,EllisActWithDrawal,CondoConversion,RoommateSameUnit,OtherCause,LatePayments,LeadRemediation,Development,GoodSamaritanEnds,Neighborhood,date
30,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,mission,2016-06-27
37,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,mission,2016-06-16
59,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,mission,2016-06-27
77,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,mission,2016-06-22
82,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,mission,2016-06-30


In [26]:
evictions_mission.to_csv('/data/leils/evictions_mission.csv', index=False, index_label=False)

# Tidying Rent Increase Data 
---
A far simpler data set, we only need to convert the dates to datetimes here. 

In [3]:
rent_increase_raw = pd.read_csv('/data/leils/Annual_Allowable_Rent_Increase_for_Units_Under_Rent_Control.csv')

In [21]:
rent_increase_raw.head()

,Effective Starting Date,Annual Allowable Percent Increase
0,04/01/1982,7.0
1,03/01/1983,7.0
2,03/01/1984,4.0
3,03/01/1985,4.0
4,03/01/1986,4.0


In [24]:
rent_increase_tidy = rent_increase_raw.rename({"Annual Allowable Percent Increase":"PercentIncrease"})
rent_increase_tidy['date'] = pd.to_datetime(rent_increase_tidy['Effective Starting Date'])
del rent_increase_tidy['Effective Starting Date']

rent_increase_tidy.head()

,Annual Allowable Percent Increase,date
0,7.0,1982-04-01
1,7.0,1983-03-01
2,4.0,1984-03-01
3,4.0,1985-03-01
4,4.0,1986-03-01


In [48]:
rent_increase_tidy.to_csv('/data/leils/tidy_rent_increase.csv', index=False, index_label=False)

# Tidying Police Incident Data 

In [5]:
police_raw = pd.read_csv('/data/leils/SFPD_Incidents_-_from_1_January_2003.csv')
police_raw.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160


In [11]:
police_tidy = police_raw[police_raw['PdDistrict'] == 'MISSION']
police_tidy = police_tidy[['Category', 'PdDistrict', 'Date']]
police_tidy['Date'] = pd.to_datetime(police_tidy['Date'])
police_tidy['PdDistrict'] = police_tidy['PdDistrict'].str.lower()

In [12]:
police_tidy.head()

,Category,PdDistrict,Date
0,NON-CRIMINAL,mission,2015-01-19
10,LARCENY/THEFT,mission,2015-02-01
11,DRUG/NARCOTIC,mission,2015-02-01
12,DRUG/NARCOTIC,mission,2015-02-01
13,WARRANTS,mission,2015-02-01


In [27]:
police_tidy.to_csv('/data/leils/tidy_police.csv', index=False, index_label=False)

In [14]:
len(police_tidy.index)

275574